# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('G:\coursera\machine_learning\course#2_regression_modeling\w2\kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to mostafaahmed456@gmail.com and will expire on July 24, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Mustafa\AppData\Local\Temp\graphlab_server_1478365057.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [14]:
all_features =  ['bedrooms',  'bathrooms']
             
              

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [19]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, l2_penalty=0., l1_penalty=1.4e8)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 2

Number of unpacked features : 2

...

| 5         | 6        | 0.000008  | 0.788526     | 6410663.529953     | 325596.397686 |

| 6         | 7        | 0.000008  | 0.837559     | 6382203.988007     | 324531.508514 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [20]:
print(model_all.get("coefficients")).print_rows(num_rows=19, num_columns=9)

+-------------+-------+---------------+--------+
|     name    | index |     value     | stderr |
+-------------+-------+---------------+--------+
| (intercept) |  None | 120536.905793 |  None  |
|   bedrooms  |  None |  42129.91411  |  None  |
|  bathrooms  |  None | 134884.417999 |  None  |
+-------------+-------+---------------+--------+
[3 rows x 4 columns]

None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [9]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [10]:
import numpy as np

l1_penalty =np.logspace(1, 7, num=13)

result_frame =graphlab.SFrame()

In [11]:
penalty_rss = []
for i in l1_penalty :
    model =graphlab.linear_regression.create(training,features =all_features ,target='price',
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=i,verbose = False)
    prdictions = model.predict(validation)
    resudials =validation['price']-prdictions
    resudials =resudials*resudials
    Rss = resudials.sum()
    
    
    print("l1_penalty: %s, RSS: $%.6f" % (i, Rss))
    penalty_rss.append((i, Rss))



l1_penalty: 10.0, RSS: $625766285142459.875000
l1_penalty: 31.6227766017, RSS: $625766285362394.125000
l1_penalty: 100.0, RSS: $625766286057885.000000
l1_penalty: 316.227766017, RSS: $625766288257224.625000
l1_penalty: 1000.0, RSS: $625766295212186.750000
l1_penalty: 3162.27766017, RSS: $625766317206080.500000
l1_penalty: 10000.0, RSS: $625766386760658.125000
l1_penalty: 31622.7766017, RSS: $625766606749278.500000
l1_penalty: 100000.0, RSS: $625767302791634.125000
l1_penalty: 316227.766017, RSS: $625769507643886.250000
l1_penalty: 1000000.0, RSS: $625776517727024.000000
l1_penalty: 3162277.66017, RSS: $625799062845467.000000
l1_penalty: 10000000.0, RSS: $625883719085425.250000


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [12]:
penalty_rss= sorted(penalty_rss, key = lambda x : x[1])
l1_penalty_best = penalty_rss[0][0]
print(l1_penalty_best)

 


10.0


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [13]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10.0,verbose=False)
model_all.get("coefficients").print_rows(num_rows=120)
 

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  20459.2475219   |  None  |
|     bedrooms     |  None |  8155.38098737   |  None  |
| bedrooms_square  |  None |  1479.73787423   |  None  |
|    bathrooms     |  None |  24576.2383172   |  None  |
|   sqft_living    |  None |  37.4911504798   |  None  |
| sqft_living_sqrt |  None |  1109.39597073   |  None  |
|     sqft_lot     |  None | -0.0168499198461 |  None  |
|  sqft_lot_sqrt   |  None |  149.569423985   |  None  |
|      floors      |  None |  20983.5137368   |  None  |
|  floors_square   |  None |  12278.1023451   |  None  |
|    waterfront    |  None |  581971.306649   |  None  |
|       view       |  None |  92988.9899686   |  None  |
|    condition     |  None |  6924.28719657   |  None  |
|      grade       |  None |  6205.64105779   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [67]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [69]:
l1_penalty_values = np.logspace(8, 10, num=20)
l1_penalty_values

array([  1.00000000e+08,   1.27427499e+08,   1.62377674e+08,
         2.06913808e+08,   2.63665090e+08,   3.35981829e+08,
         4.28133240e+08,   5.45559478e+08,   6.95192796e+08,
         8.85866790e+08,   1.12883789e+09,   1.43844989e+09,
         1.83298071e+09,   2.33572147e+09,   2.97635144e+09,
         3.79269019e+09,   4.83293024e+09,   6.15848211e+09,
         7.84759970e+09,   1.00000000e+10])

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [80]:
penalty_nnz = []
for i in l1_penalty_values :
    model =graphlab.linear_regression.create(training,features =all_features ,target='price',
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=i,verbose = False)
    weights =model.get('coefficients')['value']
    
    penalty_nnz.append((i, (model.coefficients['value']).nnz()))
     
    
  



In [81]:
penalty_nnz

[(100000000.0, 18),
 (127427498.57031322, 18),
 (162377673.91887242, 18),
 (206913808.11147901, 18),
 (263665089.87303555, 17),
 (335981828.62837881, 17),
 (428133239.8719396, 17),
 (545559478.11685145, 17),
 (695192796.17755914, 17),
 (885866790.41008317, 16),
 (1128837891.6846883, 15),
 (1438449888.2876658, 15),
 (1832980710.8324375, 13),
 (2335721469.0901213, 12),
 (2976351441.6313128, 10),
 (3792690190.7322536, 6),
 (4832930238.5717525, 5),
 (6158482110.6602545, 3),
 (7847599703.5146227, 1),
 (10000000000.0, 1)]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [85]:

l1_penalty_min = max([t[0] for t in penalty_nnz if t[1] > max_nonzeros])
l1_penalty_max = min([t[0] for t in penalty_nnz if t[1] < max_nonzeros])
print(l1_penalty_min, l1_penalty_max)

(2976351441.6313128, 3792690190.7322536)


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [86]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [88]:
l1_penalty = np.linspace(l1_penalty_min,l1_penalty_max,20)


penalty_rss = []
for i in l1_penalty :
    model =graphlab.linear_regression.create(training,features =all_features ,target='price',
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=i,verbose = False)
    prdictions = model.predict(validation)
    resudials =validation['price']-prdictions
    resudials =resudials*resudials
    Rss = resudials.sum()
    
    
    print("l1_penalty: %s, RSS: $%.6f" % (i, Rss))
    penalty_rss.append((i, Rss))



l1_penalty: 2976351441.63, RSS: $966925692362084.500000
l1_penalty: 3019316638.95, RSS: $974019450084556.125000
l1_penalty: 3062281836.27, RSS: $981188367942452.750000
l1_penalty: 3105247033.59, RSS: $989328342459474.000000
l1_penalty: 3148212230.92, RSS: $998783211265891.250000
l1_penalty: 3191177428.24, RSS: $1008477167020094.000000
l1_penalty: 3234142625.56, RSS: $1018298780553819.750000
l1_penalty: 3277107822.88, RSS: $1028247992205977.250000
l1_penalty: 3320073020.2, RSS: $1034616909232828.125000
l1_penalty: 3363038217.52, RSS: $1038554735941040.750000
l1_penalty: 3406003414.84, RSS: $1043237237871703.000000
l1_penalty: 3448968612.16, RSS: $1046937488751711.125000
l1_penalty: 3491933809.48, RSS: $1051147625612860.875000
l1_penalty: 3534899006.81, RSS: $1055992735342999.125000
l1_penalty: 3577864204.13, RSS: $1060799531763287.750000
l1_penalty: 3620829401.45, RSS: $1065707689498230.125000
l1_penalty: 3663794598.77, RSS: $1069464335425586.500000
l1_penalty: 3706759796.09, RSS: $1073

In [89]:
penalty_rss= sorted(penalty_rss, key = lambda x : x[1])
l1_penalty_best = penalty_rss[0][0]
print(l1_penalty_best)


2976351441.63


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [90]:
model.get('coefficients')

name,index,value,stderr
(intercept),None,240309.75932,None
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
bathrooms,None,13840.6399577,None
sqft_living,None,30.5583588298,None
sqft_living_sqrt,None,592.199469213,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None
